In [1]:
import pandas as pd
import numpy as np
import sys
import os
from netCDF4 import Dataset, date2num
import glob
import scipy.io
import tqdm
import xarray as xr
import os

In [2]:
import numpy as np
import pandas as pd
import os
from pomegranate import *

from scipy.stats import genextreme as gev
from scipy.stats import gumbel_l as gumbel
from scipy.stats import gamma as gamma
from scipy.stats import expon as exp
from scipy.stats import lognorm as logn
from scipy.stats import gumbel_r as gumbel_r
from scipy.stats import powerlaw as powerlaw

import warnings
import numpy as np
import pandas as pd
import scipy.stats as st
import statsmodels as sm
import statsmodels.api as sma
import matplotlib
import matplotlib.pyplot as plt
from statsmodels.distributions.empirical_distribution import ECDF
import warnings
from pyproj import Proj, transform
from statsmodels.tsa.seasonal import seasonal_decompose
from dateutil.parser import parse
warnings.filterwarnings('ignore')

import seaborn as sns
sns.set(); sns.set_context('notebook')

In [3]:
def make_pdf(dist, params, size=10000):
    """Generate distributions's Probability Distribution Function """

    # Separate parts of parameters
    arg = params[:-2]
    loc = params[-2]
    scale = params[-1]

    # Get sane start and end points of distribution
    start = dist.ppf(0.01, *arg, loc=loc, scale=scale) if arg else dist.ppf(0.01, loc=loc, scale=scale)
    end = dist.ppf(0.99, *arg, loc=loc, scale=scale) if arg else dist.ppf(0.99, loc=loc, scale=scale)

    # Build PDF and turn into pandas Series
    x = np.linspace(start, end, size)
    y = dist.pdf(x, loc=loc, scale=scale, *arg)
    pdf = pd.Series(y, x)

    return pdf

In [4]:
def Ajuste_GEV_KMA_Frechet(data, plot=False):
    #Ajuste Gumbel
    func=getattr(st,'genextreme')
    paramt = func.fit(data,fc=0.0000000001)
    nlogL = np.log(func.pdf(data,*paramt)).sum()
    parmhatgev = func.fit(data)
    nlogLGev = np.log(func.pdf(data,*parmhatgev)).sum()
    
    if -parmhatgev[0]>0:
        isgev = np.sum((np.abs(nlogLGev)-np.abs(nlogL))>=1.92)
        if  isgev==1:
                paramGev= parmhatgev
        else:
             paramGev = paramt
    else:
        paramGev= parmhatgev

    if plot == True:
        fig, ax=plt.subplots(2, 2, figsize=(12, 8))
        data.plot(kind='hist', bins=50, density =True, alpha=0.5, color='red',ax=ax[0,0])
        dataYLim = ax[0,0].get_ylim()
        distribution=getattr(st,'genextreme')
        best_distribution = distribution
        best_dist = getattr(st,'genextreme')
        best_fit_params = paramGev
        best_fit_name='genextreme'

        #----------------------------------------------------------------------------------- 
        pdf = make_pdf(best_dist, best_fit_params)

        pdf.plot(lw=2, label='PDF', legend=True, ax=ax[0,0])
        data.plot(kind='hist', bins=50, density=True, alpha=0.5, label='Data', legend=True, ax=ax[0,0])

        param_names = (best_dist.shapes + ', loc, scale').split(', ') if best_dist.shapes else ['loc', 'scale']
        param_str = ', '.join(['{}={:0.2f}'.format(k,v) for k,v in zip(param_names, best_fit_params)])
        dist_str = '{}({})'.format(best_fit_name, param_str)

        ax[0,0].set_title(u'Best probability density function \n' + dist_str)
        ax[0,0].set_xlabel(u'Data')
        ax[0,0].set_ylabel('Frequency')
        ax[0,0].set_ylim(dataYLim)
        #-----------------------------------------------------------------------------------     
        data=data.values.flatten()
        PQmax_1 = best_dist.cdf(data.astype(float),*best_fit_params);
        Qemp = np.sort(data);
        kk = np.arange(1,len(data)+1);
        prob = kk/(len(data));
        Qlogn  = best_dist.ppf(prob,*best_fit_params);
        ax[0,1].plot(Qemp,Qlogn,'.k')
        ax[0,1].plot(np.arange(min(data),max(data)),np.arange(min(data),max(data)),'-b')
        ax[0,1].set_title('Q-Q Plot \n' + dist_str)
        ax[0,1].set_xlabel(u'Empirical')
        ax[0,1].set_ylabel('Model')
        #-----------------------------------------------------------------------------------     
        ecdf=ECDF(data)
        x = np.linspace(min(data),max(data),200)
        y = best_dist.cdf(x, *best_fit_params)
        ax[1,0].plot(x, y)
        ax[1,0].scatter(data.astype(float),ecdf(data))
        ax[1,0].set_title('Best Cumulative density function \n' + dist_str)
        ax[1,0].set_xlabel(u'Data')
        ax[1,0].set_ylabel('Prob')
        #-----------------------------------------------------------------------------------    
        x_r=np.arange(0.01,1000)
        prob_t=(1-1/x_r)
        y_r=best_dist.ppf(prob_t,*best_fit_params)
        ax[1,1].plot(x_r,y_r,'-b')

        ecdf=ECDF(data)
        t_r=1/(1-ecdf(data))
        ax[1,1].plot(t_r,data,'.k')
        ax[1,1].set_xscale("log")
        ax[1,1].set_xlabel(u'Return Periods')
        ax[1,1].set_ylabel('Return Values')
        ax[1,1].set_title('Return Values Plot \n' + dist_str)

        print(best_fit_params)
        fig.tight_layout()  
    func=getattr(st,'genextreme')
    
    PeriodoRetorno=[2, 3, 4, 5, 6, 7, 8, 9, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100,200,500,1000]
    TWL = pd.DataFrame(index=PeriodoRetorno,columns=['TWL'])
    for i in PeriodoRetorno:
        TWL.loc[i] = func.ppf((1-1/i),*paramGev)
        
    return TWL

In [5]:
def matDatenum2PYDatetime(datenumVec,unitTime = 'D'):
    datetimeVec = pd.to_datetime(datenumVec-719529, unit=unitTime,errors='coerce')
    datetimeNum = datenumVec-719529
    return datetimeVec,datetimeNum

In [6]:
def create_netcf_Dinamicas_DOW(path_files,path_output,model,scenario):
    directory = list() 
    for file in os.listdir(path_files):
        if file.startswith("Punto"):
            directory.append(file)
    names_point = list()
    lon = list()
    lat = list()
    for d, dd in enumerate(directory):
        names_point.append(dd[:-19])
        lon.append(float(dd[-17:-10]))
        lat.append(float(dd[-8:-1]))
        
    for i,ii in enumerate(tqdm.tqdm(directory)):
        if i ==0:
            mat    = scipy.io.loadmat(path_files+directory[i]+'/'+directory[i]+'_'+model+'_'+scenario+'.mat')
            time   = mat['time'].flatten()
            hs_o   = mat['hs'].reshape(1,-1)
            tps_o  = mat['tps'].reshape(1,-1)
            dir_o  = mat['dir'].reshape(1,-1)
            zeta_o = mat['zeta'].reshape(1,-1)
            tm02_o = mat['tm02'].reshape(1,-1)
        else:
            mat = scipy.io.loadmat(path_files+directory[i]+'/'+directory[i]+'_'+model+'_'+scenario+'.mat')
            hs_o   = np.concatenate((hs_o, mat['hs'].reshape(1,-1)), axis=0)
            tps_o  = np.concatenate((tps_o, mat['tps'].reshape(1,-1)), axis=0)
            dir_o  = np.concatenate((dir_o, mat['dir'].reshape(1,-1)), axis=0)
            zeta_o = np.concatenate((zeta_o, mat['zeta'].reshape(1,-1)), axis=0)
            tm02_o = np.concatenate((tm02_o, mat['tm02'].reshape(1,-1)), axis=0)
        
    time_py = matDatenum2PYDatetime(time,unitTime = 'D')[0]
    
    nc = Dataset(path_output+'PtosObjtvo_DIN_'+model+'_'+scenario+'.nc', 'w', format='NETCDF4')
    # Global Attributes 
    nc.description= 'Contiene las dinámicas del modelo '+model+' en el escenario '+scenario  
    # nc dimensions
    nc.createDimension('lon',  len(lon))
    nc.createDimension('lat',  len(lat))
    nc.createDimension('time',len(time))
    #nc.createDimension('point',len(names_point))
    # crear variables
    xx_nc=nc.createVariable('lon','float32', ('lon'))
    yy_nc=nc.createVariable('lat','float32', ('lat'))
    time_nc=nc.createVariable('time','float32',('time'))
    #point_nc = nc.createVariable('point','int',('point'))
    hs_nc=nc.createVariable('hs','float32', ('time','lon'))
    tps_nc=nc.createVariable('tps','float32', ('time','lon'))
    dir_nc=nc.createVariable('dir','float32', ('time','lon'))
    zeta_nc=nc.createVariable('zeta','float32', ('time','lon'))
    tm02_nc=nc.createVariable('tm02','float32', ('time','lon'))
    #units
    xx_nc.units = 'degrees_east'
    yy_nc.units = 'degrees_north'
    time_nc.units='days since '+str(time_py[0].year)+'-01-01'
    hs_nc.units='m'
    tps_nc.units='s'
    dir_nc.units='º'
    zeta_nc.units='º'
    tm02_nc.units='s'
    #long_name
    xx_nc.long_name = 'longitude coordinate'
    yy_nc.long_name = 'latitud coordinate'
    time_nc.long_name='dias del año'
    hs_nc.long_name='Altura de ola'
    tps_nc.long_name='Período de pico suavizado'
    dir_nc.long_name='Dirección'
    zeta_nc.long_name='Marea meteorológica'
    tm02_nc.long_name='Periodo medio'
    # calendar
    time_nc.calendar = 'standard'
    # rellenar variables
    #point_nc[:]=np.arange(1,len(names_point)+1)
    xx_nc[:]=lon
    yy_nc[:]=lat
    time_nc[:]=date2num(time_py.to_pydatetime(), units='days since '+str(time_py[0].year)+'-01-01', calendar='standard')
    hs_nc[:]=hs_o[:,:].T
    tps_nc[:]=tps_o[:,:].T
    dir_nc[:]=dir_o[:,:].T
    zeta_nc[:] = zeta_o[:,:].T
    tm02_nc[:]=tm02_o[:,:].T
    nc.close()   

In [56]:
def create_netcf_MareaAst(path_files,path_output):
    directory = list() 
    for file in os.listdir(path_files):
        if file.startswith("Punto"):
            directory.append(file)
    names_point = list()
    lon = list()
    lat = list()
    for d, dd in enumerate(directory):
        names_point.append(dd[:-23])
        lon.append(float(dd[-21:-14]))
        lat.append(float(dd[-12:-5]))
        
    for i,ii in enumerate(tqdm.tqdm(directory)):
        if i ==0:
            mat    = scipy.io.loadmat(path_files+directory[i])
            time   = mat['time'].flatten()
            lon_tide_o = mat['lon_tide']
            lat_tide_o = mat['lat_tide']
            tide_o  = mat['tide'].reshape(1,-1)
            u_o  = mat['u'].reshape(1,-1)
            v_o  = mat['v'].reshape(1,-1)
        else:
            mat    = scipy.io.loadmat(path_files+directory[i])
            tide_o  =  np.concatenate((tide_o,mat['tide'].reshape(1,-1)),axis=0)
            lon_tide_o = np.concatenate((lon_tide_o,mat['lon_tide']),axis=0)
            lat_tide_o = np.concatenate((lat_tide_o,mat['lat_tide']),axis=0)
            u_o  = np.concatenate((u_o,mat['u'].reshape(1,-1)),axis=0)
            v_o  = np.concatenate((v_o,mat['v'].reshape(1,-1)),axis=0)
        
    time_py = matDatenum2PYDatetime(time,unitTime = 'D')[0]
    nc = Dataset(path_output+'PtosObjtvo_MAT.nc', 'w', format='NETCDF4')
    # Global Attributes 
    nc.description= 'Contiene la variable Marea Astronómica' 
    # nc dimensions
    nc.createDimension('lon',  len(lon))
    nc.createDimension('lat',  len(lat))
    nc.createDimension('time',len(time))
    #nc.createDimension('point',len(names_point))
    # crear variables
    xx_nc=nc.createVariable('lon','float32', ('lon'))
    yy_nc=nc.createVariable('lat','float32', ('lat'))
    time_nc=nc.createVariable('time','float32',('time'))
    lon_tide_nc = nc.createVariable('lon_tide','float32', ('lon'))
    lat_tide_nc = nc.createVariable('lat_tide','float32', ('lat'))
    #point_nc = nc.createVariable('point','int',('point'))
    tide_nc=nc.createVariable('tide','float32', ('time','lon'))
    u_nc=nc.createVariable('u','float32', ('time','lon'))
    v_nc=nc.createVariable('v','float32', ('time','lon'))
    #units
    xx_nc.units = 'degrees_east'
    yy_nc.units = 'degrees_north'
    time_nc.units='days since '+str(time_py[0].year)+'-01-01'
    lon_tide_nc.units = 'degrees_east'
    lat_tide_nc.units = 'degrees_north'
    tide_nc.units='m'
    u_nc.units='m'
    v_nc.units='m'
    #long_name
    xx_nc.long_name = 'longitude coordinate'
    yy_nc.long_name = 'latitud coordinate'
    lon_tide_nc.long_name = 'longitude coordinate tide'
    lat_tide_nc.long_name = 'latitud coordinate tide'
    time_nc.long_name='dias del año'
    tide_nc.long_name='Altura'
    u_nc.long_name='Dirección horizontal'
    v_nc.long_name='Dirección vertical'
    
    # calendar
    time_nc.calendar = 'standard'
    # rellenar variables
    #point_nc[:]=np.arange(1,len(names_point)+1)
    xx_nc[:]=lon
    yy_nc[:]=lat
    lon_tide_nc[:] = lon_tide_o
    lat_tide_nc[:] = lat_tide_o
    time_nc[:]=date2num(time_py.to_pydatetime(), units='days since '+str(time_py[0].year)+'-01-01', calendar='standard')
    tide_nc[:]=tide_o[:,:].T
    u_nc[:]=u_o[:,:].T
    v_nc[:]=v_o[:,:].T
    nc.close()

In [57]:
def create_netcf_Dinamicas_SLR(path_files,path_output):
    directory = list() 
    for file in os.listdir(path_files):
        if file.startswith("Punto"):
            directory.append(file)
    names_point = list()
    lon = list()
    lat = list()
    for d, dd in enumerate(directory):
        names_point.append(dd[:-27])
        lon.append(float(dd[-25:-18]))
        lat.append(float(dd[-16:-9]))
        
    for i,ii in enumerate(tqdm.tqdm(directory)):
        if i ==0:
            mat    = scipy.io.loadmat(path_files+directory[i])
            time   = mat['time'].flatten()
            m_ensemble45_o  = mat['m_ensemble45'].reshape(1,-1)
            m_ensemble85_o  = mat['m_ensemble85'].reshape(1,-1)
            P5_45_o  = mat['P5_45'].reshape(1,-1)
            P5_85_o  = mat['P5_85'].reshape(1,-1)
            P95_45_o = mat['P95_45'].reshape(1,-1)
            P95_85_o = mat['P95_85'].reshape(1,-1)
        else:
            mat = scipy.io.loadmat(path_files+directory[i])
            m_ensemble45_o   = np.concatenate((m_ensemble45_o, mat['m_ensemble45'].reshape(1,-1)), axis=0)
            m_ensemble85_o  = np.concatenate((m_ensemble85_o, mat['m_ensemble85'].reshape(1,-1)), axis=0)
            P5_45_o  = np.concatenate((P5_45_o, mat['P5_45'].reshape(1,-1)), axis=0)
            P5_85_o  = np.concatenate((P5_85_o, mat['P5_85'].reshape(1,-1)), axis=0)
            P95_45_o = np.concatenate((P95_45_o, mat['P95_45'].reshape(1,-1)), axis=0)
            P95_85_o = np.concatenate((P95_85_o, mat['P95_85'].reshape(1,-1)), axis=0)
        
    time_py = matDatenum2PYDatetime(time,unitTime = 'D')[0]
    
    nc = Dataset(path_output+'PtosObjtvo_MSL.nc', 'w', format='NETCDF4')
    # Global Attributes 
    nc.description= 'Contiene la variable SLR' 
    # nc dimensions
    nc.createDimension('lon',  len(lon))
    nc.createDimension('lat',  len(lat))
    nc.createDimension('time',len(time))
    #nc.createDimension('point',len(names_point))
    # crear variables
    xx_nc=nc.createVariable('lon','float32', ('lon'))
    yy_nc=nc.createVariable('lat','float32', ('lat'))
    time_nc=nc.createVariable('time','float32',('time'))
    #point_nc = nc.createVariable('point','int',('point'))
    m_ensemble45_nc=nc.createVariable('m_ensemble45','float32', ('time','lon'))
    m_ensemble85_nc=nc.createVariable('m_ensemble85','float32', ('time','lon'))
    P5_45_nc=nc.createVariable('P5_45','float32', ('time','lon'))
    P5_85_nc=nc.createVariable('P5_85','float32', ('time','lon'))
    P95_45_nc=nc.createVariable('P95_45','float32', ('time','lon'))
    P95_85_nc=nc.createVariable('P95_85','float32', ('time','lon'))
    #units
    xx_nc.units = 'degrees_east'
    yy_nc.units = 'degrees_north'
    time_nc.units='days since '+str(time_py[0].year)+'-01-01'
    m_ensemble45_nc.units='m'
    m_ensemble85_nc.units='s'
    P5_45_nc.units='º'
    P5_85_nc.units='º'
    P95_45_nc.units='º'
    P95_85_nc.units='l'
    #long_name
    xx_nc.long_name = 'longitude coordinate'
    yy_nc.long_name = 'latitud coordinate'
    time_nc.long_name='dias del año'
    m_ensemble45_nc.long_name='Ensemble RCP 45'
    m_ensemble85_nc.long_name='Ensemble RCP 85'
    P5_45_nc.long_name='Percentil 5% para el RCP 45'
    P5_85_nc.long_name='Percentil 5% para el RCP 85'
    P95_45_nc.long_name='Percentil 95% para el RCP 45'
    P95_85_nc.long_name='Percentil 95% para el RCP 85'
    # calendar
    time_nc.calendar = 'standard'
    # rellenar variables
    #point_nc[:]=np.arange(1,len(names_point)+1)
    xx_nc[:]=lon
    yy_nc[:]=lat
    time_nc[:]=date2num(time_py.to_pydatetime(), units='days since '+str(time_py[0].year)+'-01-01', calendar='standard')
    m_ensemble45_nc[:]=m_ensemble45_o[:,:].T
    m_ensemble85_nc[:]=m_ensemble85_o[:,:].T
    P5_45_nc[:]=P5_45_o[:,:].T
    P5_85_nc[:]=P5_85_o[:,:].T
    P95_45_nc[:] = P95_45_o[:,:].T
    P95_85_nc[:]=P95_85_o[:,:].T
    nc.close()

In [99]:
def setup_fun (Tp,Hs,idplaya,slope,CStock):
            L0=9.81/(2.*np.pi)*(Tp**2);
            if idplaya == 1:
                beta = slope; #% 1/30;
                Irb=beta/np.sqrt(Hs/L0);#% Batjes,1974
                # Stockdon,H.F. 2006; Runup2%
                Stockdon_setup=CStock*beta*np.sqrt(Hs*L0);
                #        s = find(Irb < 0.3);
                #         Stockdon_setup(s)=0.016.*sqrt(Hs(s).*L0(s));
                #Stockdon_setup=0.04.*sqrt(Hs.*L0);
            else:
                Stockdon_setup=0.08*np.sqrt(Hs*L0);
            return Stockdon_setup
        

In [362]:
def generate_files_extrems_CC(path_inputs_dinamicas,model,path_project,table_point_carac,refNMMA,fin_year_period,cod_epsg):
    """ La siguiente función nos permite generar los ficheros de TWL para cada escenario de cambio climático.
        A continuación se explican cada uno de los parámetros de entrada necesarios y los outputs generados
        
        Parámetros:
        ---------------------
        path_inputs_dinamicas   : string. path donde se encuentran los netcdf de las dinámicas para cada uno de los puntos de estudio
        model                   : string. Modelo de cambio climático que se desea analizar
        path_project            : string. path donde se encuentra el proyecto de RFSM
        table_point_carac       : pandas dataframe. Dataframe con las coordenadas y características de los puntos donde se van a asignar las dinámicas
        refNMMA                 : float. Referencia nivel medio del mar
        fin_year_period         : array. Fecha de fin de cada uno de los períodos que se desean analizar
        cod_epsg                : int. Código epsg del sistema de coordenadas utilizado en las coordendas de los puntos de estudio
        
        Salidas:
        ---------------------
        Ficheros csv para cada uno de modelos, escenarios y periodos.
    
    """

    PtosObjtvo_DOW_his_nc = xr.open_dataset(path_inputs_dinamicas+'PtosObjtvo_DIN_'+model+'_his.nc')
    PtosObjtvo_DOW_his_nc = PtosObjtvo_DOW_his_nc.sortby('lon')
    PtosObjtvo_DOW_rcp45_nc = xr.open_dataset(path_inputs_dinamicas+'PtosObjtvo_DIN_'+model+'_rcp45.nc')
    PtosObjtvo_DOW_rcp45_nc = PtosObjtvo_DOW_rcp45_nc.sortby('lon')
    PtosObjtvo_DOW_rcp85_nc = xr.open_dataset(path_inputs_dinamicas+'PtosObjtvo_DIN_'+model+'_rcp85.nc')
    PtosObjtvo_DOW_rcp85_nc = PtosObjtvo_DOW_rcp85_nc.sortby('lon')
    PtosObjtvo_MSL_nc = xr.open_dataset(path_inputs_dinamicas+'PtosObjtvo_MSL.nc')
    PtosObjtvo_MSL_nc = PtosObjtvo_MSL_nc.sortby('lon')
    PtosObjtvo_MAT_nc = xr.open_dataset(path_inputs_dinamicas+'PtosObjtvo_MAT.nc')
    PtosObjtvo_MAT_nc = PtosObjtvo_MAT_nc.sortby('lon')

    PeriodoRetorno=[2, 3, 4, 5, 6, 7, 8, 9, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100,200,500,1000]
    string_PR = [str(int) for int in PeriodoRetorno]
    if os.path.exists(path_project+'sites/CC/')==False:
        os.mkdir(path_project+'sites/CC/')
    
    for t in range(len(fin_year_period)):
        print('Calculando TWL del período hasta '+str(fin_year_period[t]))
        TWL_extrem_rcp_45_tab = pd.DataFrame(index=table_point_carac.index,columns=string_PR+['MSLR','SLR_5','SLR_95'])
        TWL_extrem_rcp_85_tab = pd.DataFrame(index=table_point_carac.index,columns=string_PR+['MSLR','SLR_5','SLR_95'])
        
        TWL_extrem_hist_tab = pd.DataFrame(index=table_point_carac.index,columns=string_PR+['MSLR','SLR_5','SLR_95'])
        TWL_extrem_hist_tab.loc[:,['MSLR','SLR_5','SLR_95']] = 0
   
        for i in tqdm.tqdm(range(len(table_point_carac))):

            x_0 = table_point_carac.iloc[i,0]
            y_0 = table_point_carac.iloc[i,1]

            inProj = Proj(init='epsg:'+str(cod_epsg))
            outProj = Proj(init='epsg:4326')

            x,y = transform(inProj,outProj,x_0,y_0)


            idoleaje = table_point_carac.iloc[i,2]
            idmarea = table_point_carac.iloc[i,3]
            idplaya = table_point_carac.iloc[i,3]
            slope = table_point_carac.iloc[i,4]
            CStock = table_point_carac.iloc[i,5]

            point_selec_his = PtosObjtvo_DOW_his_nc.point.data[np.sqrt((x-PtosObjtvo_DOW_his_nc.lon.data)**2+(y-PtosObjtvo_DOW_his_nc.lat.data)**2).argmin()]
            
            point_selec_rcp_45 = PtosObjtvo_DOW_rcp45_nc.point.data[np.sqrt((x-PtosObjtvo_DOW_rcp45_nc.lon.data)**2+(y-PtosObjtvo_DOW_rcp45_nc.lat.data)**2).argmin()]
            
            point_selec_rcp_85 = PtosObjtvo_DOW_rcp85_nc.point.data[np.sqrt((x-PtosObjtvo_DOW_rcp85_nc.lon.data)**2+(y-PtosObjtvo_DOW_rcp85_nc.lat.data)**2).argmin()]
            
            point_selec_MSL = PtosObjtvo_MSL_nc.point.data[np.sqrt((x-PtosObjtvo_MSL_nc.lon.data)**2+(y-PtosObjtvo_MSL_nc.lat.data)**2).argmin()]
            
            point_selec_MAT = PtosObjtvo_MAT_nc.point.data[np.sqrt((x-PtosObjtvo_MAT_nc.lon.data)**2+(y-PtosObjtvo_MAT_nc.lat.data)**2).argmin()]
            
            
            
            Hs_his = PtosObjtvo_DOW_his_nc.sel(point = point_selec_his)['hs'].to_dataframe()
            Hs_his.index = Hs_his.index.round('H')
            try:
                Hs_his[Hs_his.hs<0]=0.001
            except:
                Hs_his = Hs_his

            Tp_his = PtosObjtvo_DOW_his_nc.sel(point = point_selec_his)['tps'].to_dataframe()
            Tp_his.index = Tp_his.index.round('H')
            try:
                Tp_his[Tp_his.tps<=0]=np.mean(Tp_his[Tp_his>0],axis=0)
            except:
                Tp_his = Tp_his

            SS_his = PtosObjtvo_DOW_his_nc.sel(point = point_selec_his)['zeta'].to_dataframe()
            SS_his.index = SS_his.index.round('H')

            Hs_rcp_45 = PtosObjtvo_DOW_rcp45_nc.sel(point = point_selec_rcp_45)['hs'].to_dataframe()
            Hs_rcp_45.index = Hs_rcp_45.index.round('H')
            try:
                Hs_rcp_45[Hs_rcp_45.hs<0]=0.001
            except:
                Hs_rcp_45 = Hs_rcp_45

            Tp_rcp_45 = PtosObjtvo_DOW_rcp45_nc.sel(point = point_selec_rcp_45)['tps'].to_dataframe()
            Tp_rcp_45.index = Tp_rcp_45.index.round('H')
            try:
                Tp_rcp_45[Tp_rcp_45.tps<=0]=np.mean(Tp_rcp_45[Tp_rcp_45>0],axis=0)
            except:
                Tp_rcp_45 = Tp_rcp_45

            SS_rcp45 = PtosObjtvo_DOW_rcp45_nc.sel(point = point_selec_rcp_45)['zeta'].to_dataframe()
            SS_rcp45.index = SS_rcp45.index.round('H')

            Hs_rcp_85 = PtosObjtvo_DOW_rcp85_nc.sel(point = point_selec_rcp_85)['hs'].to_dataframe()
            Hs_rcp_85.index = Hs_rcp_85.index.round('H')
            try:
                Hs_rcp_85[Hs_rcp_85.hs<0]=0.001
            except:
                Hs_rcp_85 = Hs_rcp_85

            Tp_rcp_85 = PtosObjtvo_DOW_rcp85_nc.sel(point = point_selec_rcp_85)['tps'].to_dataframe()
            Tp_rcp_85.index = Tp_rcp_85.index.round('H')
            try:
                Tp_rcp_85[Tp_rcp_85.tps<=0]=np.mean(Tp_rcp_85[Tp_rcp_85>0],axis=0)
            except:
                Tp_rcp_85 = Tp_rcp_85

            SS_rcp85 = PtosObjtvo_DOW_rcp85_nc.sel(point = point_selec_rcp_85)['zeta'].to_dataframe()
            SS_rcp85.index = SS_rcp85.index.round('H')


            MSLR_rcp45 = PtosObjtvo_MSL_nc.sel(point = point_selec_MSL)['m_ensemble45'].to_dataframe()
            MSLR_rcp45.index = MSLR_rcp45.index.round('H')

            SLR_95_rcp45   = PtosObjtvo_MSL_nc.sel(point = point_selec_MSL)['P95_45'].to_dataframe()
            SLR_95_rcp45.index = SLR_95_rcp45.index.round('H')

            SLR_5_rcp45   = PtosObjtvo_MSL_nc.sel(point = point_selec_MSL)['P5_45'].to_dataframe()
            SLR_5_rcp45.index = SLR_5_rcp45.index.round('H')


            MSLR_rcp85 = PtosObjtvo_MSL_nc.sel(point = point_selec_MSL)['m_ensemble85'].to_dataframe()
            MSLR_rcp85.index = MSLR_rcp85.index.round('H')

            SLR_95_rcp85   = PtosObjtvo_MSL_nc.sel(point = point_selec_MSL)['P95_85'].to_dataframe()
            SLR_95_rcp85.index = SLR_95_rcp85.index.round('H')

            SLR_5_rcp85   = PtosObjtvo_MSL_nc.sel(point = point_selec_MSL)['P5_85'].to_dataframe()
            SLR_5_rcp85.index = SLR_5_rcp85.index.round('H')

            AT = PtosObjtvo_MAT_nc.sel(point = point_selec_MAT)['tide'].to_dataframe()
            AT.index = AT.index.round('H')

            PMVE =AT.iloc[AT.values.argmax()-3:AT.values.argmax()+4]

            setup_hist  = pd.DataFrame(index=Tp_his.index, columns=['setup'])
            setup_rcp45 = pd.DataFrame(index=Tp_rcp_45.index, columns=['setup'])
            setup_rcp85 = pd.DataFrame(index=Tp_rcp_85.index, columns=['setup'])

            setup_hist.iloc[:,0]  = setup_fun (Tp_his.values,Hs_his.values*idoleaje,idplaya,slope,CStock)
            setup_rcp45.iloc[:,0] = setup_fun (Tp_rcp_45.values,Hs_rcp_45.values*idoleaje,idplaya,slope,CStock)
            setup_rcp85.iloc[:,0] = setup_fun (Tp_rcp_85.values,Hs_rcp_45.values*idoleaje,idplaya,slope,CStock)

            ## Cálculo de la cota de inundación / TWL

            Result_hist  = pd.concat((setup_hist,SS_his,AT),axis = 1).dropna()
            Result_rcp45 = pd.concat((setup_rcp45,SS_rcp45,AT),axis=1).dropna()
            Result_rcp85 = pd.concat((setup_rcp85,SS_rcp85,AT),axis=1).dropna()

            TWL_hist  = Result_hist.iloc[:,0]+Result_hist.iloc[:,1]*idmarea+Result_hist.iloc[:,2]*idmarea+refNMMA
            TWL_rcp45 = Result_rcp45.iloc[:,0]+Result_rcp45.iloc[:,1]*idmarea+Result_rcp45.iloc[:,2]*idmarea+refNMMA
            TWL_rcp85 = Result_rcp85.iloc[:,0]+Result_rcp85.iloc[:,1]*idmarea+Result_rcp85.iloc[:,2]*idmarea+refNMMA

            
            TWL_rcp_45 = pd.concat((TWL_hist,TWL_rcp45.loc[:'2040'])).resample('A').max().dropna().astype(float)
            TWL_rcp_85 = pd.concat((TWL_hist,TWL_rcp85.loc[:'2040'])).resample('A').max().dropna().astype(float)
            
            TWL_extrem_rcp_45 = Ajuste_GEV_KMA_Frechet(TWL_rcp_45,plot=False)
            TWL_extrem_rcp_85 = Ajuste_GEV_KMA_Frechet(TWL_rcp_85,plot=False)
            
            MSLR_RCP45   = MSLR_rcp45.loc[:str(fin_year_period[t])].values[-1][0]
            MSLR_RCP85   = MSLR_rcp85.loc[:str(fin_year_period[t])].values[-1][0]
            SLR_5_RCP45  = SLR_5_rcp45.loc[:str(fin_year_period[t])].values[-1][0]
            SLR_5_RCP85  = SLR_5_rcp85.loc[:str(fin_year_period[t])].values[-1][0]
            SLR_95_RCP45 = SLR_95_rcp45.loc[:str(fin_year_period[t])].values[-1][0]
            SLR_95_RCP85 = SLR_95_rcp85.loc[:str(fin_year_period[t])].values[-1][0]
            
            TWL_extrem_rcp_45_tab.iloc[i,:len(PeriodoRetorno)]  = (TWL_extrem_rcp_45.values).reshape(1,-1)
            TWL_extrem_rcp_85_tab.iloc[i,:len(PeriodoRetorno)]  = (TWL_extrem_rcp_85.values).reshape(1,-1)
            
            
            TWL_extrem_rcp_45_tab.iloc[i,len(PeriodoRetorno):]  = [MSLR_RCP45, SLR_5_RCP45, SLR_95_RCP45]
            TWL_extrem_rcp_85_tab.iloc[i,len(PeriodoRetorno):]  = [MSLR_RCP85, SLR_5_RCP85, SLR_95_RCP85]
            
        
            TWL_extrem_his       = Ajuste_GEV_KMA_Frechet(TWL_hist.resample('A').max().dropna().astype(float),plot=False)
            TWL_extrem_hist_tab.iloc[i,:len(PeriodoRetorno)] = (TWL_extrem_his.values).reshape(1,-1)
            
        TWL_extrem_rcp_45_tab.to_csv(path_project+'sites/CC/'+'TWL_extrem_'+model+'_RCP45_'+str(fin_year_period[t])+'.csv')
        TWL_extrem_rcp_85_tab.to_csv(path_project+'sites/CC/'+'TWL_extrem_'+model+'_RCP85_'+str(fin_year_period[t])+'.csv')
        TWL_extrem_hist_tab.to_csv(path_project+'sites/CC/'+'TWL_extrem_'+model+'_his.csv')

In [ ]:
def generate_files_extrems(path_inputs_dinamicas,model,path_project,table_point_carac,refNMMA,fin_year_period,cod_epsg):
    """ La siguiente función nos permite generar los ficheros de TWL para cada escenario de cambio climático.
        A continuación se explican cada uno de los parámetros de entrada necesarios y los outputs generados
        
        Parámetros:
        ---------------------
        path_inputs_dinamicas   : string. path donde se encuentran los netcdf de las dinámicas para cada uno de los puntos de estudio
        model                   : string. Modelo de cambio climático que se desea analizar
        path_project            : string. path donde se encuentra el proyecto de RFSM
        table_point_carac       : pandas dataframe. Dataframe con las coordenadas y características de los puntos donde se van a asignar las dinámicas
        refNMMA                 : float. Referencia nivel medio del mar
        fin_year_period         : array. Fecha de fin de cada uno de los períodos que se desean analizar
        cod_epsg                : int. Código epsg del sistema de coordenadas utilizado en las coordendas de los puntos de estudio
        
        Salidas:
        ---------------------
        Ficheros csv para cada uno de modelos, escenarios y periodos.
    
    """
    PtosObjtvo_ROW_his_nc = xr.open_dataset(path_inputs_dinamicas+'PtosObjtvo_DIN_ROW_his.nc')
    PtosObjtvo_MAT_nc = xr.open_dataset(path_inputs_dinamicas+'PtosObjtvo_MAT.nc')

    PeriodoRetorno=[2, 3, 4, 5, 6, 7, 8, 9, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100,200,500,1000]
    string_PR = [str(int) for int in PeriodoRetorno]
    
    print('Calculando TWL del período hasta '+str(fin_year_period[t]))

    TWL_extrem_hist_tab = pd.DataFrame(index=table_point_carac.index,columns=string_PR+['MSLR','SLR_5','SLR_95'])
    TWL_extrem_hist_tab.loc[:,['MSLR','SLR_5','SLR_95']] = 0

    for i in tqdm.tqdm(range(len(table_point_carac))):

        x_0 = table_point_carac.iloc[i,0]
        y_0 = table_point_carac.iloc[i,1]

        inProj = Proj(init='epsg:'+str(cod_epsg))
        outProj = Proj(init='epsg:4326')

        x,y = transform(inProj,outProj,x_0,y_0)


        idoleaje = table_point_carac.iloc[i,2]
        idmarea = table_point_carac.iloc[i,3]
        idplaya = table_point_carac.iloc[i,3]
        slope = table_point_carac.iloc[i,4]
        CStock = table_point_carac.iloc[i,5]

        point_selec_his = PtosObjtvo_ROW_his_nc.point.data[np.sqrt((x-PtosObjtvo_ROW_his_nc.lon.data)**2+(y-PtosObjtvo_ROW_his_nc.lat.data)**2).argmin()]

        point_selec_MAT = PtosObjtvo_MAT_nc.point.data[np.sqrt((x-PtosObjtvo_MAT_nc.lon.data)**2+(y-PtosObjtvo_MAT_nc.lat.data)**2).argmin()]



        Hs_his = PtosObjtvo_ROW_his_nc.sel(point = point_selec_his)['hs'].to_dataframe()
        Hs_his.index = Hs_his.index.round('H')
        try:
            Hs_his[Hs_his.hs<0]=0.001
        except:
            Hs_his = Hs_his

        Tp_his = PtosObjtvo_ROW_his_nc.sel(point = point_selec_his)['tps'].to_dataframe()
        Tp_his.index = Tp_his.index.round('H')
        try:
            Tp_his[Tp_his.tps<=0]=np.mean(Tp_his[Tp_his>0],axis=0)
        except:
            Tp_his = Tp_his

        SS_his = PtosObjtvo_ROW_his_nc.sel(point = point_selec_his)['zeta'].to_dataframe()
        SS_his.index = SS_his.index.round('H')


        AT = PtosObjtvo_MAT_nc.sel(point = point_selec_MAT)['tide'].to_dataframe()
        AT.index = AT.index.round('H')

        PMVE =AT.iloc[AT.values.argmax()-3:AT.values.argmax()+4]

        setup_hist  = pd.DataFrame(index=Tp_his.index, columns=['setup'])

        setup_hist.iloc[:,0]  = setup_fun (Tp_his.values,Hs_his.values*idoleaje,idplaya,slope,CStock)

        ## Cálculo de la cota de inundación / TWL

        Result_hist  = pd.concat((setup_hist,SS_his,AT),axis = 1).dropna()

        TWL_hist  = Result_hist.iloc[:,0]+Result_hist.iloc[:,1]*idmarea+Result_hist.iloc[:,2]*idmarea+refNMMA

        TWL_extrem_his       = Ajuste_GEV_KMA_Frechet(TWL_hist.resample('A').max().dropna().astype(float),plot=False)
        TWL_extrem_hist_tab.iloc[i,:len(PeriodoRetorno)] = (TWL_extrem_his.values).reshape(1,-1)
            
        TWL_extrem_hist_tab.to_csv(path_project+'sites/'+'TWL_extrem_'+model+'_his.csv')

In [373]:
def generate_files_extrems(path_inputs_dinamicas,path_project,table_point_carac,refNMMA,cod_epsg):
    """ La siguiente función nos permite generar los ficheros de TWL para el escenario ROW.
        A continuación se explican cada uno de los parámetros de entrada necesarios y los outputs generados
        
        Parámetros:
        ---------------------
        path_inputs_dinamicas   : string. path donde se encuentran los netcdf de las dinámicas para cada uno de los puntos de estudio
        path_project            : string. path donde se encuentra el proyecto de RFSM
        table_point_carac       : pandas dataframe. Dataframe con las coordenadas y características de los puntos donde se van a asignar las dinámicas
        refNMMA                 : float. Referencia nivel medio del mar
        cod_epsg                : int. Código epsg del sistema de coordenadas utilizado en las coordendas de los puntos de estudio    
        
        Salidas:
        ---------------------
        Ficheros csv para cada uno de modelos, escenarios y periodos.
    
    """
    PtosObjtvo_ROW_his_nc = xr.open_dataset(path_inputs_dinamicas+'PtosObjtvo_DIN_ROW_his.nc')
    PtosObjtvo_MAT_nc = xr.open_dataset(path_inputs_dinamicas+'PtosObjtvo_MAT.nc')

    PeriodoRetorno=[2, 3, 4, 5, 6, 7, 8, 9, 10, 20,25, 30, 40, 50, 60, 70, 80, 90, 100,200,500,1000]
    string_PR = [str(int) for int in PeriodoRetorno]
    
    print('Calculando TWL')

    TWL_extrem_hist_tab = pd.DataFrame(index=table_point_carac.index,columns=string_PR+['MSLR','SLR_5','SLR_95'])
    TWL_extrem_hist_tab.loc[:,['MSLR','SLR_5','SLR_95']] = 0
    
    Params_extrem_his    = pd.DataFrame(index=table_point_carac.index,columns=['c','loc','scale'])

    for i in tqdm.tqdm(range(len(table_point_carac))):

        x_0 = table_point_carac.iloc[i,0]
        y_0 = table_point_carac.iloc[i,1]

        inProj = Proj(init='epsg:'+str(cod_epsg))
        outProj = Proj(init='epsg:4326')

        x,y = transform(inProj,outProj,x_0,y_0)


        idoleaje = table_point_carac.iloc[i,2]
        idmarea = table_point_carac.iloc[i,3]
        idplaya = table_point_carac.iloc[i,3]
        slope = table_point_carac.iloc[i,4]
        CStock = table_point_carac.iloc[i,5]

        point_selec_his = PtosObjtvo_ROW_his_nc.point.data[np.sqrt((x-PtosObjtvo_ROW_his_nc.lon.data)**2+(y-PtosObjtvo_ROW_his_nc.lat.data)**2).argmin()]

        point_selec_MAT = PtosObjtvo_MAT_nc.point.data[np.sqrt((x-PtosObjtvo_MAT_nc.lon.data)**2+(y-PtosObjtvo_MAT_nc.lat.data)**2).argmin()]



        Hs_his = PtosObjtvo_ROW_his_nc.sel(point = point_selec_his,drop=True)['hs'].to_dataframe()
        Hs_his.index = Hs_his.index.round('H')
        try:
            Hs_his[Hs_his.hs<0]=0.001
        except:
            Hs_his = Hs_his

        Tp_his = PtosObjtvo_ROW_his_nc.sel(point = point_selec_his,drop=True)['tps'].to_dataframe()
        Tp_his.index = Tp_his.index.round('H')
        try:
            Tp_his[Tp_his.tps<=0]=np.mean(Tp_his[Tp_his>0],axis=0)
        except:
            Tp_his = Tp_his

        SS_his = PtosObjtvo_ROW_his_nc.sel(point = point_selec_his,drop=True)['zeta'].to_dataframe()
        SS_his.index = SS_his.index.round('H')


        AT = PtosObjtvo_MAT_nc.sel(point = point_selec_MAT,drop=True)['tide'].to_dataframe()
        AT.index = AT.index.round('H')

        PMVE =AT.iloc[AT.values.argmax()-3:AT.values.argmax()+4]

        setup_hist  = pd.DataFrame(index=Tp_his.index, columns=['setup'])

        setup_hist.iloc[:,0]  = setup_fun (Tp_his.values,Hs_his.values*idoleaje,idplaya,slope,CStock)

        ## Cálculo de la cota de inundación / TWL

        Result_hist  = pd.concat((setup_hist,SS_his,AT),axis = 1).dropna()

        TWL_hist  = Result_hist.iloc[:,0]+Result_hist.iloc[:,1]*idmarea+Result_hist.iloc[:,2]*idmarea+refNMMA
        

        TWL_hist  = Result_hist.iloc[:,0]+Result_hist.iloc[:,1]*idmarea+Result_hist.iloc[:,2]*idmarea+refNMMA
           
        TWL_extrem_his = Ajuste_GEV_KMA_Frechet(TWL_hist.resample('A').max().dropna().astype(float),path_project,save_plot=False,name_fig=None)[0]
              
        TWL_extrem_hist_tab.iloc[i,:len(PeriodoRetorno)]   = TWL_extrem_his.values.reshape(1,-1)
        
        Params_extrem_his.iloc[i,:]     = Ajuste_GEV_KMA_Frechet(TWL_hist.resample('A').max().dropna().astype(float),path_project,save_plot=True,name_fig='Params_extrem_'+'_ROW_'+'_POINT_'+str(i))[1:]
              
    TWL_extrem_hist_tab.to_csv(path_project+'sites/'+'TWL_extrem_ROW.csv')
    Params_extrem_his.to_csv(path_project+'sites/'+'Params_extrem_ROW.csv')

In [ ]:
def tusrBCFlowLevel(table_points,file_TWL,temporal,path_project,TestDesc,Table_impact_zone,
                     izcoast,BCSetID,BCTypeID_COAST,return_period,SLR,
                    raw_q=True, point_X_river = None ,point_Y_river = None ,hidrograma=None):
    
    """La siguiente función nos permite generar los ficheros de condición de contorno de RFSM.
    
       Parámetros:
       ---------------------
       table_points       : pandas dataframe. Dataframe con las coordenadas y características de los puntos donde asignaron las dinámicas
       file_TWL           : pandas dataframe. Dataframe que contiene los puntos donde se extraen los períodos de retrono
       temporal           : pandas dataframe. Dataframe que contiene el temporal de referencia.
       path_project       : string. path donde se encuentra el proyecto de RFSM
       TestDesc           : string. Nombre de la simulación
       Table_impact_zone  : pandas dataframe.  Dataframe con el conjunto de impact zones
       izcoast            : pandas dataframe.  Dataframe con el conjunto de impact zones donde se asigna condición de contorno costera
       BCSetID            : int. Identificador de la simulación
       BCTypeID_COAST     : int. Tipo de condición de contorno # 1 overtopping; # 2 level;
       return_period      : int. Período de retorno estudiado
       SLR                : float: Sea Level Rise
       raw_q              : True or False: Si existe un punto donde se va introducir un caudal en un cauce poner en True
       point_X_river      : float. Si existe un punto donde se va introducir un caudal en un cauce añadir 
       point_Y_river      : float. Si existe un punto donde se va introducir un caudal en un cauce poner en True
       hidrograma         : array. Si existe un punto donde se va introducir un caudal en un cauce añadir hidrograma 
       
        Salidas:
        ---------------------
        Fichero tusrBCFlowLevel necesario para la ejecución de RFSM.
       
       
       
        """
    
    Table_impact_zone_edit =Table_impact_zone.copy()
    Table_impact_zone_edit.loc[:,'BCTypeID'] = 0
    
    cI = temporal
    cI = [cI[0]*1/10,cI[0]*3/10,cI[0]*5/10,cI[0]*7/10,cI,cI[-1]*7/10,cI[-1]*5/10,cI[-1]*3/10,cI[-1]*1/10]
   
    n_periods = len(cI)
    
    Table_impact_zone_edit.loc[izcoast.index,'BCTypeID'] = BCTypeID_COAST
    
    if raw_q==True:
        
        dist = np.sqrt((point_X_river-Table_impact_zone_edit.iloc[:,2])**2+(point_Y_river-Table_impact_zone_edit.iloc[:,3])**2)
        point_select=dist.idxmin()
        
        Table_impact_zone_edit.loc[point_select,'BCTypeID'] = 10
        
    Results_TWL = pd.DataFrame(index=np.arange(0,sum(Table_impact_zone_edit.BCTypeID.values>0)*n_periods),columns=['BCSetID', 'BCTypeID', 'IZID', 'Time', 'BCValue'])
    
    Table_impact_zone_edit_2=Table_impact_zone_edit[Table_impact_zone_edit.BCTypeID!=0].copy()
    
    it=0
    for iDZ in tqdm.tqdm(range(len(Table_impact_zone_edit_2))):
        if (Table_impact_zone_edit_2['BCTypeID'].iloc[iDZ]==2) or (Table_impact_zone_edit_2['BCTypeID'].iloc[iDZ]==1):

            x_pointIZID = Table_impact_zone_edit_2[' MidX'].values[iDZ]
            y_pointIZID = Table_impact_zone_edit_2[' MidY'].values[iDZ]
            IZID = Table_impact_zone_edit_2.index[iDZ]

            dist = np.sqrt((x_pointIZID-table_points.iloc[:,0])**2+(y_pointIZID-table_points.iloc[:,1])**2)
            point_select=dist.idxmin()

            
            TWL=table_points.loc[return_period,table_points.iloc[point_select]]
            
            cITr = cI/max(cI)*TWL
            pos1= np.where(cITr == np.max(cITr))[0]

            cITr[pos1]= cITr[pos1] + SLR
            
            if BCTypeID_COAST ==1:
                cITr = cITr*izcoast.nCells[IZID]*cellsize
            else:
                cITr = cITr-izcoast.minH[IZID]
                cITr[cITr<0]=0
                
            

            Results_TWL.iloc[it:it+n_periods,0] = BCSetID
            Results_TWL.iloc[it:it+n_periods,1] = BCTypeID_COAST
            Results_TWL.iloc[it:it+n_periods,2] = IZID
            Results_TWL.iloc[it:it+n_periods,3] = np.linspace(0,3600*len(cITr),len(cITr))
            Results_TWL.iloc[it:it+n_periods,4] = cITr

            it=it+n_periods

            del W, MM, MA, Hs, setup, TWL

        elif Table_impact_zone_edit_2['BCTypeID'].iloc[iDZ]==10:
            print('Río')
            Results_TWL.iloc[it:it+n_periods,0] = BCSetID
            Results_TWL.iloc[it:it+n_periods,1] = 1
            Results_TWL.iloc[it:it+n_periods,2] = IZID
            Results_TWL.iloc[it:it+n_periods,3] = np.linspace(0,3600*len(hidrograma),len(hidrograma))
            Results_TWL.iloc[it:it+n_periods,4] = hidrograma

            it=it+n_periods
            
    Results_TWL.to_csv(path_project+'tests/'+TestDesc+'/Input_User/tusrBCFlowLevel.csv',index=False)

In [36]:
def create_netcf_Dinamicas_DOW(path_files,path_output,model,scenario):
    directory = list() 
    for file in os.listdir(path_files):
        if file.startswith("Punto"):
            directory.append(file)
    names_point = list()
    lon = list()
    lat = list()
    
    
    
    for d, dd in enumerate(directory):
        pos_ = [*find_all(dd, '[')]
        pos_r = [*find_all(dd, ']')]
        
        names_point.append(dd[:pos_[0]-1])
        lon.append(float(dd[pos_[0]+1:pos_r[0]]))
        lat.append(float(dd[pos_[1]+1:pos_r[1]]))  
        
    for i,ii in enumerate(tqdm.tqdm(directory)):
        if model == 'DOW2' and scenario == 'his':
            path_file_ = path_files+directory[i]+'/'+directory[i]+'_'+'DOW2'+'.mat'
        else:
            path_file_ = path_files+directory[i]+'/'+directory[i]+'_'+model+'_'+scenario+'.mat'
        
        if i ==0:
            mat    = scipy.io.loadmat(path_file_)
            time   = mat['time'].flatten()
            hs_o   = mat['hs'].reshape(1,-1)
            tps_o  = mat['tps'].reshape(1,-1)
            dir_o  = mat['dir'].reshape(1,-1)
            zeta_o = mat['zeta'].reshape(1,-1)
            tm02_o = mat['tm02'].reshape(1,-1)
            bat_o = mat['bat']
        else:
            try:
                mat = scipy.io.loadmat(path_file_)
            except:
                print('Error en el fichero: '+ path_file_)
                continue
            hs_o   = np.concatenate((hs_o, mat['hs'].reshape(1,-1)), axis=0)
            tps_o  = np.concatenate((tps_o, mat['tps'].reshape(1,-1)), axis=0)
            dir_o  = np.concatenate((dir_o, mat['dir'].reshape(1,-1)), axis=0)
            zeta_o = np.concatenate((zeta_o, mat['zeta'].reshape(1,-1)), axis=0)
            tm02_o = np.concatenate((tm02_o, mat['tm02'].reshape(1,-1)), axis=0)
            bat_o = np.concatenate((bat_o, mat['bat']), axis=0)
        
    time_py = matDatenum2PYDatetime(time,unitTime = 'D')[0]
    
    nc = Dataset(path_output+'PtosObjtvo_DIN_'+model+'_'+scenario+'.nc', 'w', format='NETCDF4')
    # Global Attributes 
    nc.description= 'Contiene las dinámicas del modelo '+model+' en el escenario '+scenario  
    # nc dimensions
    nc.createDimension('lon',  len(lon))
    nc.createDimension('lat',  len(lat))
    nc.createDimension('time',len(time))
    nc.createDimension('point',len(names_point))
    # crear variables
    xx_nc=nc.createVariable('lon','float32', ('lon'))
    yy_nc=nc.createVariable('lat','float32', ('lat'))
    time_nc=nc.createVariable('time','float32',('time'))
    point_nc = nc.createVariable('point','int',('point'))
    hs_nc=nc.createVariable('hs','float32', ('time','point'))
    tps_nc=nc.createVariable('tps','float32', ('time','point'))
    dir_nc=nc.createVariable('dir','float32', ('time','point'))
    zeta_nc=nc.createVariable('zeta','float32', ('time','point'))
    tm02_nc=nc.createVariable('tm02','float32', ('time','point'))
    bat_nc = nc.createVariable('bat','float32',('point'))
    #units
    xx_nc.units = 'degrees_east'
    yy_nc.units = 'degrees_north'
    time_nc.units='days since '+str(time_py[0].year)+'-01-01'
    hs_nc.units='m'
    tps_nc.units='s'
    dir_nc.units='º'
    zeta_nc.units='º'
    tm02_nc.units='s'
    point_nc.units = ''
    bat_nc.units = 'm'
    #long_name
    xx_nc.long_name = 'longitude coordinate'
    yy_nc.long_name = 'latitud coordinate'
    time_nc.long_name='dias del año'
    hs_nc.long_name='Altura de ola'
    tps_nc.long_name='Período de pico suavizado'
    dir_nc.long_name='Dirección'
    zeta_nc.long_name='Marea meteorológica'
    tm02_nc.long_name='Periodo medio'
    point_nc.long_name = 'Puntos de estudio'
    bat_nc.long_name = 'Profundidad'
    # calendar
    time_nc.calendar = 'standard'
    # rellenar variables
    point_nc[:]=np.arange(1,len(names_point)+1)
    xx_nc[:]=lon
    yy_nc[:]=lat
    time_nc[:]=date2num(time_py.to_pydatetime(), units='days since '+str(time_py[0].year)+'-01-01', calendar='standard')
    hs_nc[:]=hs_o[:,:].T
    tps_nc[:]=tps_o[:,:].T
    dir_nc[:]=dir_o[:,:].T
    zeta_nc[:] = zeta_o[:,:].T
    tm02_nc[:]=tm02_o[:,:].T
    bat_nc[:]=bat_o[:].T
    nc.close()   